In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Lambda
import subprocess as sp
import numpy as np

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(path='mnist.npz')


In [3]:
num_samples = x_train.shape[0]
num_classes = 10

In [4]:
BATCH_SIZE = 256
ds_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
ds_train = ds_train.map(lambda x, y: (tf.reshape(x, [-1])/255, tf.one_hot(y, num_classes)))
ds_train = ds_train.batch(BATCH_SIZE).repeat().prefetch(1)

ds_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
ds_test = ds_test.map(lambda x, y: (tf.reshape(x, [-1])/255, tf.one_hot(y, num_classes)))
ds_test = ds_test.batch(BATCH_SIZE).repeat().prefetch(1)



In [5]:
tf.keras.backend.clear_session()

memory_usage = []
class MemoryCheck(tf.keras.callbacks.Callback):
    def on_train_batch_end(self, batch, logs=None):
        mem = sp.check_output('nvidia-smi | grep python', shell=True).split()[-2].decode('utf-8')
        memory_usage.append(int(mem[:-3]))
        print(' ' + mem)

mem_check = MemoryCheck()


In [6]:
@tf.function
def cube_this(x):
    return x**3

In [7]:
from layers import Highway

model = Sequential()
model.add(Dense(512, activation = 'relu', input_shape = (784, )))
model.add(Dropout(0.2))
model.add(Highway())
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation = 'softmax'))
model.build()

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
highway (Highway)            (None, 512)               525312    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                32832     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 960,714
Trainable params: 960,714
Non-trainable params: 0
__________________________________________________

In [9]:
model.compile('RMSprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [10]:
model.fit_generator(generator = ds_train, 
                    steps_per_epoch = int(np.ceil(num_samples /float(BATCH_SIZE))), 
                    validation_data = ds_test, 
                    validation_steps = int(np.ceil(x_test.shape[0] /float(BATCH_SIZE))),
                    
                    epochs = 10, callbacks = [mem_check])

Epoch 1/10
 911MiB
 87/235 [==========>...................] - ETA: 10s - loss: 0.5787 - accuracy: 0.8212 911MiB


175/235 [=====================>........] - ETA: 4s - loss: 0.4193 - accuracy: 0.8715 911MiB


235/235 [==============================] - 17s 73ms/step - loss: 0.3611 - accuracy: 0.8895 - val_loss: 0.2593 - val_accuracy: 0.9251
Epoch 2/10
 911MiB
 28/235 [==>...........................] - ETA: 15s - loss: 0.1653 - accuracy: 0.9530 911MiB


 58/235 [======>.......................] - ETA: 12s - loss: 0.1678 - accuracy: 0.9509

KeyboardInterrupt: 